# Homework：汽车销售聊天机器人

## 使用 GPT-4 构造汽车销售话术数据



使用 ChatGPT 构造销售数据的 Prompt ：

```
你是中国顶级的汽车销售，现在培训职场新人，请给出100条实用的销售话术。
每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt.txt](real_estate_sales_data.txt.txt) 文件中

In [23]:
with open("real_estate_sales_data.txt", encoding="UTF-8") as f:
    car_sale_data = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [24]:
from langchain.text_splitter import CharacterTextSplitter

In [25]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [26]:
docs = text_splitter.create_documents([car_sale_data])

Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100


In [27]:
docs[0]

Document(page_content='[客户问题] 这款车的价格有点高，有没有优惠？\n[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。')

In [28]:
len(docs)

41

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [30]:
query = "这款车的价格有点高，有没有优惠？"

In [31]:
answer_list = db.similarity_search(query)

In [32]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这款车的价格有点高，有没有优惠？
[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。

[客户问题] 这款车有现车吗？
[销售回答] 目前这款车的库存情况还不错，不过由于这款车的市场需求量很大，建议您尽快下单，我们可以为您预留车辆，确保您第一时间提到车。

[客户问题] 这款车的保养费用高吗？
[销售回答] 我们的保养费用非常合理，并且我们有多个保养套餐供您选择，能有效控制您的维护成本。此外，定期保养可以延长车辆的使用寿命，减少意外维修的机会。

[客户问题] 这款车的零配件贵吗？
[销售回答] 这款车的零配件价格非常合理，并且我们提供原厂配件，保证质量和性能。定期保养和正确使用车辆可以有效减少零配件的更换频率，降低维护成本。



In [33]:
db.save_local("car_sale_data")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [34]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [35]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C87D5BE320>, search_kwargs={'k': 3})

In [36]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的价格有点高，有没有优惠？
[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。

[客户问题] 这款车有现车吗？
[销售回答] 目前这款车的库存情况还不错，不过由于这款车的市场需求量很大，建议您尽快下单，我们可以为您预留车辆，确保您第一时间提到车。

[客户问题] 这款车的保养费用高吗？
[销售回答] 我们的保养费用非常合理，并且我们有多个保养套餐供您选择，能有效控制您的维护成本。此外，定期保养可以延长车辆的使用寿命，减少意外维修的机会。



In [37]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [38]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车有现车吗？
[销售回答] 目前这款车的库存情况还不错，不过由于这款车的市场需求量很大，建议您尽快下单，我们可以为您预留车辆，确保您第一时间提到车。

[客户问题] 你们的贷款购车条件是什么？
[销售回答] 我们提供多种灵活的贷款方案，首付比例低至20%，最长贷款期限可达5年。只需提供身份证、收入证明和居住证明，我们的金融顾问会为您量身定制最适合您的贷款方案。

[客户问题] 这款车的价格有点高，有没有优惠？
[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [39]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [40]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的价格有点高，有没有优惠？
[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。

[客户问题] 这款车有现车吗？
[销售回答] 目前这款车的库存情况还不错，不过由于这款车的市场需求量很大，建议您尽快下单，我们可以为您预留车辆，确保您第一时间提到车。

[客户问题] 这款车的保养费用高吗？
[销售回答] 我们的保养费用非常合理，并且我们有多个保养套餐供您选择，能有效控制您的维护成本。此外，定期保养可以延长车辆的使用寿命，减少意外维修的机会。



### 提取向量数据库中的`销售回答`

In [41]:
docs = retriever.get_relevant_documents(query)

In [42]:
docs[0].page_content

'[客户问题] 这款车的价格有点高，有没有优惠？\n[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。'

In [43]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这款车的价格有点高，有没有优惠？\n',
 '我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。']

In [44]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [45]:
ans

'我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。'

#### 尝试各种问题

In [46]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [47]:
query = "我想离医院近点"

print(sales(query))

[]


D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [48]:
print(sales(query, 0.75))

[]


D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [49]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。', '这款车的二手保值率非常高，得益于其良好的品牌口碑和出色的品质。无论是驾驶体验还是市场需求，这款车都表现非常出色，保值率也自然水涨船高。', '这款车的零配件价格非常合理，并且我们提供原厂配件，保证质量和性能。定期保养和正确使用车辆可以有效减少零配件的更换频率，降低维护成本。', '我们提供多种灵活的贷款方案，首付比例低至20%，最长贷款期限可达5年。只需提供身份证、收入证明和居住证明，我们的金融顾问会为您量身定制最适合您的贷款方案。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [50]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [51]:
qa_chain({"query": "你们小区有200万的房子吗？"})

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？',
 'result': '对不起，作为一个AI，我没有具体的地理位置信息或房地产列表的访问权限。如果您想了解某个小区是否有200万的房子，建议您联系当地的房地产经纪人，或者在网上搜索相关房产信息网站，如链家、贝壳找房等，以获取最新的房源信息和价格。'}

In [52]:
qa_chain({"query": "小区吵不吵"})

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '小区吵不吵',
 'result': '对不起，我不知道这个小区的具体情况，所以无法回答这个问题。如果你能提供更多信息，我会尽力帮助你。'}

In [53]:
print(sales("小区吵不吵"))

[]


D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [54]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [55]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [56]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么',
 'result': '对不起，作为一个人工智能，我无法提供购买别墅的服务。我在这里主要是为了提供信息和帮助解答问题。'}

In [57]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [58]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '对不起，我是一个人工智能助手，我无法提供购买别墅的服务。我可以帮助回答问题或提供信息，但无法进行实物交易。'}

In [59]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [60]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\project\openai-quickstart\venv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [61]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮您购买别墅。我是一个人工智能，用来帮助回答问题和提供信息，但我不能进行实际的交易或购买。',
 'source_documents': []}